In [1]:
!pip install -q wikipedia nltk

In [2]:
import wikipedia

def fetch_wiki(topic):
    wikipedia.set_lang("en")
    try:
        page = wikipedia.page(topic, auto_suggest=False)
        return page.summary
    except wikipedia.DisambiguationError as e:
        return wikipedia.summary(e.options[0], sentences=3)
    except:
        return "No information found"


print(fetch_wiki("Cat"))

The cat (Felis catus), also called domestic cat and house cat, is a small carnivorous mammal. It is an obligate carnivore, requiring a predominantly meat-based diet. Its retractable claws are adapted to killing small prey species such as mice and rats. It has a strong, flexible body, quick reflexes, and sharp teeth, and its night vision and sense of smell are well developed. It is a social species, but a solitary hunter and a crepuscular predator.
Cat communication includes meowing, purring, trilling, hissing, growling, grunting, and body language. It can hear sounds too faint or too high in frequency for human ears, such as those made by small mammals. It secretes and perceives pheromones. Cat intelligence is evident in its ability to adapt, learn through observation, and solve problems. 
Female domestic cats can have kittens from spring to late autumn in temperate zones and throughout the year in equatorial regions, with litter sizes often ranging from two to five kittens.
The domest

In [3]:
import nltk
nltk.download("punkt")
nltk.download("punkt_tab")


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [4]:
import nltk
nltk.download("punkt")
from nltk.tokenize import sent_tokenize

def split_into_claims(answer):
    return sent_tokenize(answer)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [5]:
answer = "Alan Turing invented the computer in 1936. He was British."
claims = split_into_claims(answer)
print(claims)

['Alan Turing invented the computer in 1936.', 'He was British.']


In [6]:
claim = "Alan Turing invented the computer in 1936."
evidence = fetch_wiki("Alan Turing")

print("CLAIM:")
print(claim)
print("\nEVIDENCE:")
print(evidence)

CLAIM:
Alan Turing invented the computer in 1936.

EVIDENCE:
Alan Mathison Turing (; 23 June 1912 – 7 June 1954) was an English mathematician, computer scientist, logician, cryptanalyst, philosopher and theoretical biologist. He was highly influential in the development of theoretical computer science, providing a formalisation of the concepts of algorithm and computation with the Turing machine, which can be considered a model of a general-purpose computer. Turing is widely considered to be the father of theoretical computer science.
Born in London, Turing was raised in southern England. He graduated from King's College, Cambridge, and in 1938, earned a doctorate degree from Princeton University. During World War II, Turing worked for the Government Code and Cypher School at Bletchley Park, Britain's codebreaking centre that produced Ultra intelligence. He led Hut 8, the section responsible for German naval cryptanalysis. Turing devised techniques for speeding the breaking of German c

In [7]:
!pip install -q sentence-transformers

In [8]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("all-MiniLM-L6-v2")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading weights:   0%|          | 0/103 [00:00<?, ?it/s]

BertModel LOAD REPORT from: sentence-transformers/all-MiniLM-L6-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


In [9]:
def get_embedding(text):
    return model.encode(text)

In [10]:
from numpy import dot
from numpy.linalg import norm

def similarity(a, b):
    return dot(a, b) / (norm(a) * norm(b))


In [11]:
s1 = "Alan Turing was a pioneer of computer science."
s2 = "Alan Turing invented the computer."
s3 = "Bananas are yellow fruits."

e1 = get_embedding(s1)
e2 = get_embedding(s2)
e3 = get_embedding(s3)

print("Similar meaning:", similarity(e1, e2))
print("Different meaning:", similarity(e1, e3))


Similar meaning: 0.9034769
Different meaning: 0.07374215


In [12]:
!pip install -q faiss-cpu


In [13]:
def chunk_text(text, chunk_size=2):
    sentences = text.split(". ")
    chunks = []
    for i in range(0, len(sentences), chunk_size):
        chunk = ". ".join(sentences[i:i+chunk_size])
        if chunk.strip():
            chunks.append(chunk)
    return chunks


In [14]:
import faiss
import numpy as np

class VectorSearch:
    def __init__(self, texts):
        self.texts = texts
        self.embeddings = model.encode(texts)
        dim = self.embeddings.shape[1]
        self.index = faiss.IndexFlatL2(dim)
        self.index.add(np.array(self.embeddings))

    def search(self, query, k=1):
        query_emb = model.encode([query])
        _, indices = self.index.search(np.array(query_emb), k)
        return [self.texts[i] for i in indices[0]]


In [15]:
wiki_text = fetch_wiki("Alan Turing")
chunks = chunk_text(wiki_text)

search_engine = VectorSearch(chunks)

claim = "Alan Turing invented the computer in 1936."
result = search_engine.search(claim)

print("CLAIM:")
print(claim)
print("\nBEST EVIDENCE FOUND:")
print(result[0])


CLAIM:
Alan Turing invented the computer in 1936.

BEST EVIDENCE FOUND:
Turing was in large part responsible not only for the concept of computers, incisive theorems about their powers, and a clear vision of the possibility of computer minds, but also for the cracking of German ciphers during the Second World War. It is fair to say we owe much to Alan Turing for the fact that we are not under Nazi rule today.”




In [16]:
!pip install -q transformers torch


In [17]:
from transformers import pipeline

nli_model = pipeline(
    "text-classification",
    model="facebook/bart-large-mnli"
)



Loading weights:   0%|          | 0/515 [00:00<?, ?it/s]

In [18]:
def verify_claim_with_evidence(claim, evidence):
    text = f"{evidence} </s></s> {claim}"
    result = nli_model(text)[0]
    return result["label"], result["score"]


In [19]:
evidence = fetch_wiki("Alan Turing")

claim = "Alan Turing invented the computer in 1936."
label, confidence = verify_claim_with_evidence(claim, evidence)

print(label, confidence)


neutral 0.8561193943023682


In [20]:
claim = "Alan Turing was a British mathematician."
label, confidence = verify_claim_with_evidence(claim, evidence)

print(label, confidence)

entailment 0.9113209247589111


ENTAILMENT → good (supports truth)

NEUTRAL → suspicious

CONTRADICTION → bad (hallucination)


0 = very grounded

1 = very hallucinated

In [21]:
def hallucination_score(results):
    total = 0
    for label, confidence in results:
        if label.lower() == "entailment":
            total += 0
        elif label.lower() == "neutral":
            total += 0.5 * confidence
        elif label.lower() == "contradiction":
            total += 1.0 * confidence
    return min(total / len(results), 1)



In [22]:
def verdict_from_score(score):
    if score < 0.3:
        return "Grounded"
    elif score < 0.6:
        return "Partially Grounded"
    else:
        return "Hallucinated"


In [23]:
def normalize_claim(claim, subject):
    pronouns = ["He", "She", "They", "he", "she", "they"]
    words = claim.split()
    if words[0] in pronouns:
        return subject + " " + " ".join(words[1:])
    return claim


In [24]:
def check_answer(answer, subject):
    claims = split_into_claims(answer)

    wiki_text = fetch_wiki(subject)
    if wiki_text == "No information found":
        return "Unknown", 1.0, []

    chunks = chunk_text(wiki_text)
    search_engine = VectorSearch(chunks)

    claim_results = []

    for claim in claims:
        clean_claim = normalize_claim(claim, subject)
        if "normalize_nationality" in globals():
          clean_claim = normalize_nationality(clean_claim)


        evidence = search_engine.search(clean_claim)[0]
        label, confidence = verify_claim_with_evidence(clean_claim, evidence)

        claim_results.append({
            "claim": claim,
            "label": label,
            "confidence": confidence,
            "evidence": evidence
        })

    score = hallucination_score(
        [(c["label"], c["confidence"]) for c in claim_results]
    )
    verdict = verdict_from_score(score)

    return verdict, score, claim_results


In [ ]:
answer = input("Paste the AI-generated answer:\n")
subject = input("\nWhat is this answer about? (subject):\n")

verdict, score, details = check_answer(answer, subject)

print("\nFINAL VERDICT:", verdict)
print("HALLUCINATION SCORE:", round(score, 2))

print("\nCLAIM-BY-CLAIM RESULTS:")
for d in details:
    print("-", d["claim"])
    print("  →", d["label"], f"({d['confidence']:.2f})")
